In [7]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np


In [8]:
data = {
    'english': ['hello', 'how are you', 'thank you'],
    'dutch': ['hallo', 'hoe gaat het', 'dank je wel']
}
df = pd.DataFrame(data)


In [9]:
def tokenize(texts):
    tokenizer = Tokenizer(filters='', lower=True)
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)
    word_index = tokenizer.word_index
    return sequences, word_index, tokenizer

input_texts = df['english']
target_texts = ['<start> ' + t + ' <end>' for t in df['dutch']]

input_seq, input_word_index, input_tokenizer = tokenize(input_texts)
target_seq, target_word_index, target_tokenizer = tokenize(target_texts)


In [10]:
max_encoder_seq_length = max(len(seq) for seq in input_seq)
max_decoder_seq_length = max(len(seq) for seq in target_seq)

encoder_input_data = pad_sequences(input_seq, maxlen=max_encoder_seq_length, padding='post')
decoder_input_data = pad_sequences([seq[:-1] for seq in target_seq], maxlen=max_decoder_seq_length-1, padding='post')
decoder_target_data = pad_sequences([seq[1:] for seq in target_seq], maxlen=max_decoder_seq_length-1, padding='post')


In [11]:
embedding_dim = 64
lstm_units = 64

# Encoder
encoder_inputs = Input(shape=(None,), name="encoder_input")
encoder_embedding_layer = Embedding(len(input_word_index) + 1, embedding_dim, name="encoder_embedding")
enc_emb = encoder_embedding_layer(encoder_inputs)
encoder_lstm = LSTM(lstm_units, return_state=True, name="encoder_lstm")
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,), name="decoder_input")
decoder_embedding_layer = Embedding(len(target_word_index) + 1, embedding_dim, name="decoder_embedding")
dec_emb = decoder_embedding_layer(decoder_inputs)
decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True, name="decoder_lstm")
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(len(target_word_index) + 1, activation='softmax', name="decoder_dense")
decoder_outputs = decoder_dense(decoder_outputs)

# Final model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, None)]       0           []                               
                                                                                                  
 decoder_input (InputLayer)     [(None, None)]       0           []                               
                                                                                                  
 encoder_embedding (Embedding)  (None, None, 64)     384         ['encoder_input[0][0]']          
                                                                                                  
 decoder_embedding (Embedding)  (None, None, 64)     640         ['decoder_input[0][0]']          
                                                                                              

In [12]:
decoder_target_data = np.expand_dims(decoder_target_data, -1)

model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=2,
    epochs=300,
    verbose=1
)


Epoch 1/300
2/2 [==============================] - 3s 11ms/step - loss: 2.3025
Epoch 2/300
2/2 [==============================] - 0s 5ms/step - loss: 2.2910
Epoch 3/300
2/2 [==============================] - 0s 6ms/step - loss: 2.2818
Epoch 4/300
2/2 [==============================] - 0s 5ms/step - loss: 2.2728
Epoch 5/300
2/2 [==============================] - 0s 5ms/step - loss: 2.2624
Epoch 6/300
2/2 [==============================] - 0s 5ms/step - loss: 2.2531
Epoch 7/300
2/2 [==============================] - 0s 5ms/step - loss: 2.2416
Epoch 8/300
2/2 [==============================] - 0s 6ms/step - loss: 2.2284
Epoch 9/300
2/2 [==============================] - 0s 6ms/step - loss: 2.2167
Epoch 10/300
2/2 [==============================] - 0s 6ms/step - loss: 2.1993
Epoch 11/300
2/2 [==============================] - 0s 6ms/step - loss: 2.1817
Epoch 12/300
2/2 [==============================] - 0s 5ms/step - loss: 2.1654
Epoch 13/300
2/2 [==============================] - 0s 5ms/s

KeyboardInterrupt: 

In [17]:
encoder_model = Model(encoder_inputs, encoder_states)


In [18]:
# Inference decoder inputs
decoder_state_input_h = Input(shape=(lstm_units,))
decoder_state_input_c = Input(shape=(lstm_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_inputs = Input(shape=(None,))

# Reuse decoder embedding layer
dec_emb2 = decoder_embedding_layer(decoder_inputs)

# Reuse decoder LSTM and Dense
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_states2 = [state_h2, state_c2]

# Inference model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)


In [23]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_word_index['<start>']  # Or 'start' if that's your token

    stop_condition = False
    decoded_sentence = []

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = [word for word, index in target_word_index.items() if index == sampled_token_index][0]

        if sampled_word == '<end>' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True
        else:
            decoded_sentence.append(sampled_word)

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return ' '.join(decoded_sentence)


In [24]:
def encode_input_text(text):
    seq = input_tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_encoder_seq_length, padding='post')
    return padded

test_sentence = "how are you"
input_seq = encode_input_text(test_sentence)
translated = decode_sequence(input_seq)
print("English:", test_sentence)
print("Dutch:", translated)


1/1 [==============================] - 0s 19ms/step
English: how are you
Dutch: hoe gaat het


https://chatgpt.com/c/67e80ee8-8914-8009-b378-b9afa7150bde